# Test Order Service and Models
- This notebook allows experimentation and testing  of the Order components 

<span style="color:red; font-weight:bold">Note:</span> Remember to Restart the Notebook **if you have changed code**.<br>The notebook does not auto reload.


In [1]:
# System level imports
import sys
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
TEST_DEV_PRJ_PATH = os.environ.get("TEST_DEV_PRJ_PATH")

# ##### SET SYS PATH TO WHERE THE CODE IS. #####
# Note: Putting our code first in the sys path will make sure it gets picked up
sys.path.insert(0, f"{TEST_DEV_PRJ_PATH}/backend/src")
print(sys.path)

['/Users/dan/Code/Python/Ken/route-optimize/backend/src', '/Users/dan/miniforge3/envs/py3.13/lib/python313.zip', '/Users/dan/miniforge3/envs/py3.13/lib/python3.13', '/Users/dan/miniforge3/envs/py3.13/lib/python3.13/lib-dynload', '', '/Users/dan/Code/Python/Ken/route-optimize/.venv/lib/python3.13/site-packages']


In [2]:
# What order data are we looking at?
import os
print(os.getcwd())
relative_path = "../../data/daily_orders.xlsx"
full_path = os.path.abspath(relative_path)
print(full_path)

/Users/dan/Code/Python/Ken/route-optimize/backend/notebooks
/Users/dan/Code/Python/Ken/route-optimize/data/daily_orders.xlsx


In [3]:
# Get the services that we will use
from wulfs_routing_api.services.order_services import OrderService
from wulfs_routing_api.models.orders.supabase_order import OrderModel
from wulfs_routing_api.models.customers.supabase_customer import SupabaseCustomer
from wulfs_routing_api.services.customer_service import CustomerService

customer_service = CustomerService(SupabaseCustomer())
order_service = OrderService(OrderModel())

✅ Supabase client initialized.


In [4]:
# Test the load_orders

daily_orders_path = "../../data/daily_orders.xlsx" 
daily_orders_df = order_service.load_orders(daily_orders_path)
daily_orders_df

,customer_name,order_id,notes,name_key
0,89 Charles,,,89 charles
1,Bar Volpe,,,bar volpe
2,Fox & The Knife,,,fox the knife
3,Dryft Revere,,,dryft revere
4,Mila's Revere,,,milas revere
...,...,...,...,...
104,Cosmica,,,cosmica
105,The Beehive,,,the beehive
106,"Wilson Farm, Inc.",,,wilson farm inc
107,Zuma-Boston,,,zumaboston


In [ ]:
# Get the customer master 
# Note: This could be mocked later as we do not actually want to test customer
customer_master_df = customer_service.load_customer_master_data()
customer_master_df

In [ ]:
orders_w_customers_df, missing_customers_df = order_service.customer_details_for_orders(daily_orders_df, customer_master_df)
orders_w_customers_df.head()


In [ ]:
print(f"Number of Missing Customer Details for Daily Orders {len(missing_customers_df)}")
missing_customers_df.head()

In [ ]:
# Save this result to be used by the VRP Solver
# Lets put it in ../data for now
orders_w_customers_df.to_csv("../../data/orders_w_customers.csv",index=False)